In [3]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [4]:
batch_size = 64
# batch size 64
train_dataset = datasets.MNIST(root='./',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

# train dataset Tensor 형태로 가져오기. 학습용, 없으면 download
test_dataset =datasets.MNIST(root='./',
                            train=False,
                            transform=transforms.ToTensor())
# test dataset Tensor 형태로 가져오기.

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
# train data batch size 만큼 읽어오기
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)
# test data batch size 만큼 읽어오기

In [5]:
class Net(nn.Module):
    # Network
    def __init__(self):
        super(Net, self).__init__()
        # 부모 초기화
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # convolution layer. input 1, output 10, kernel 5*5 (10개의 서로 다른 5*5 필터를 통해서 새로운
        # 결과를 만드는 layer)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # convolution layer. input 10, output 20, kernel 5*5
        self.mp = nn.MaxPool2d(2)
        # maxpooling 2*2
        self.fc1 = nn.Linear(320, 10)
        # full connected input 320, output 10

    def forward(self, x):
        in_size = x.size(0)
        # 들어오는 사이즈 저장
        x = F.relu(self.mp(self.conv1(x)))
        # convoultion layer -> maxpooling -> relu
        x = F.relu(self.mp(self.conv2(x)))
        # convoultion layer -> maxpooling -> relu
        x = x.view(in_size, -1)
        # size matching
        x = self.fc1(x)
        # fully connected layer
        #F.log_softmax(x)
        return x

In [6]:
def train(epoch, mutual_model, mutual_optim):
    for i in range(model_num):
        mutual_model[i].train()
    # 모델 훈련
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        outputs = []
        for model in mutual_model:
            outputs.append(model(data))

        for i in range(model_num):
            ce_loss = criterion(outputs[i], target)
            kl_loss = 0

            for j in range(model_num):
                if i != j:  # model이 달라졌을 때,
                    kl_loss += loss_kl(F.log_softmax(outputs[i]),
                                       F.softmax(Variable(outputs[j])))
            loss = ce_loss + kl_loss / (model_num - 1)
            mutual_optim[i].zero_grad()
            loss.backward(retain_graph=True)
            mutual_optim[i].step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item()))

In [7]:
def test(mutual_model):
    # 모델 test
    test_loss = []
    correct = []
    for i in range(model_num):
        test_loss.append(0)
        correct.append(0)
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data), Variable(target)
            outputs = []
            for i in range(model_num):
                model = mutual_model[i]
                model.eval()
                outputs.append(model(data))
            for i in range(model_num):
                ce_loss = criterion(outputs[i], target)
                kl_loss = 0
                for j in range(model_num):
                    if i != j:
                        kl_loss += loss_kl(F.log_softmax(outputs[i]),
                                           F.softmax(Variable(outputs[j])))
                loss = ce_loss + kl_loss / (model_num - 1)

                test_loss[i] += loss
                pred = outputs[i].data.max(1, keepdim=True)[1]
                correct[i] += pred.eq(target.data.view_as(pred)).cpu().sum()

    for i in range(model_num):
        print("model",i, test_loss[i].item()/(10000/64), "&", correct[i].item(),"/",len(test_loader.dataset))

In [8]:
criterion = nn.CrossEntropyLoss()
model_num = 2
mutual_model = []
mutual_optim = []

for i in range(model_num):
    model = Net()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    mutual_model.append(model)
    mutual_optim.append(optimizer)

loss_kl = nn.KLDivLoss(reduction='batchmean')

for epoch in range(1, 10):
    train(epoch, mutual_model, mutual_optim)
test(mutual_model)
# 스토케스틱 경사하강법. learning rate 0.01, momentum 0.5

/tmp/ipykernel_23424/1146304772.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  kl_loss += loss_kl(F.log_softmax(outputs[i]),
/tmp/ipykernel_23424/1146304772.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(Variable(outputs[j])))


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.279965
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.628080
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.658096
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.448386
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.450951
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.243637
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.298286
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.508240
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.208097
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.179902
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.218564
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.182349
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.146192
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.176664
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.240261
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.276129
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.239612
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.191883
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.096268
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.125373
Train Epoch:

/tmp/ipykernel_23424/2822534600.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  kl_loss += loss_kl(F.log_softmax(outputs[i]),
/tmp/ipykernel_23424/2822534600.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(Variable(outputs[j])))


model 0 0.05861712036132812 & 9862 / 10000
model 1 0.0622461669921875 & 9851 / 10000
